In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Preprocessing (fast fourier transform)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
# raw_data = pd.read_csv("/content/drive/MyDrive/maintain/train_data1200/train_data1.csv",header=None)
import glob
path = '/content/drive/MyDrive/maintain/train_data600/'
labelss = pd.read_csv('/content/drive/MyDrive/label.csv',header = None)
labelss
# list_files = [os.path.basename(i) for i in glob.glob(path600+'*.csv', recursive = True)]

## Try preprocess

In [ ]:
# from imblearn.under_sampling import ClusterCentroids
# cc = ClusterCentroids(random_state=0)
# X_resampled, y_resampled = cc.fit_resample(X, y)

In [ ]:
d = pd.read_csv(path+'train_data29000.csv',header=None)
d

In [ ]:
a=np.stack((np.array(d[0]),np.array(d[1]),np.array(d[2])))
a=np.transpose(a,[1,0])
a.shape

In [ ]:
len(a[0])

In [ ]:
import numpy as np
from numpy.fft import fft, ifft, fftfreq
from scipy import signal
from scipy.signal import find_peaks

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(d[0])
plt.xlabel('time (s)')
plt.ylabel('m/s^2')

In [ ]:
plt.figure(figsize=(10, 8));
for i in range(3):
  plt.subplot(3, 1, i+1)
  plt.plot(d[i])
  plt.xlabel('time (s)')
  plt.ylabel('m/s^2')
plt.show()

In [ ]:
sr = 51200
# sampling interval
ts = 1.0/sr
N=12500
ts*N

In [ ]:
# sampling rate
sr = 51200
# sampling interval
ts = 1.0/sr
# t = np.arange(0,1,ts)
#data
# x = bandpasskub(d[0] , d[1]  , d[2])[0]
x=d[0]
X = fft(x)
N = len(X)
freq = fftfreq(N, 1 / sr)
# n = np.arange(N)
# T = N/sr
# freq = n/T 
plt.figure(figsize = (12, 6))
plt.subplot(4,1,1)
plt.plot(freq , np.abs(X))
plt.title('fft_plot')
# plt.xlim(0, 200)
# plt.ylim(0,200)

plt.subplot(4,1,2)
n = len(x)
fhat = np.fft.fft(x, n) #computes the fft
psd = fhat * np.conj(fhat)/n
freq2 = (1/(ts*n)) * np.arange(n)
plt.plot(freq2 ,psd)
# plt.xlim(0, 200)
# plt.ylim(0,10)

plt.subplot(4,1,3)
X2 = fft(signal.resample(x, 1000))
freq3 = fftfreq(len(X2), 1 / 1000)
plt.plot(freq3 ,X2)
plt.xlim(0, 200)

In [ ]:
from scipy.signal import butter, lfilter
from scipy.signal import freqs
import numpy as np
from numpy.fft import fft, ifft, fftfreq , irfft , rfft
from scipy import signal
from scipy.signal import find_peaks
def butter_lowpass(cutoff, fs, order):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y
def bandpasskub(data1,data2,data3):
  cutoff = 200
  fs = 51200
  order = 5
  A=butter_lowpass_filter(data1, cutoff, fs, order)
  B=butter_lowpass_filter(data2, cutoff, fs, order)
  C=butter_lowpass_filter(data3, cutoff, fs, order)
  x=np.stack((np.array(A),np.array(B),np.array(C)))
  return  x
def fourier(data):
  # sr = 2000/((1/51200)*12500)
  sr = 51200
  x = data
  N = len(x)
  X = abs(fft(x))
  freq = fftfreq(X.size, 1/sr)

  print(len(freq))
  # X[(freq>200)] = -99
  # id = X>=0
  return X[:N//2] , freq[:N//2]
  
#t=(1/51200)*12500   ,  (1/sr)*N=t
# 2000/t = sr
# a,b = fourier(d[0])
# plt.plot(b,a)
# plt.xlim(0,2000)
# plt.ylim(0,200)

In [ ]:
a,b = fourier(d[0])
# a=a[0:6250]
# b=b[0:6250]
# a = a[b>0]
# id = b>0
# b = b[id]
fig = plt.figure(figsize=(10, 5))
plt.plot(b,a )
plt.xlabel('freq (hz)')
plt.ylabel('Amplitude')

# plt.xlim(0,300)
# plt.ylim(0,200)

In [ ]:
from tqdm import tqdm
x = []
y = []
z = []
labels = []
for i in tqdm(range(len(labelss.iloc[:14500]))):
  d = pd.read_csv(path + labelss.iloc[i][0] , header = None)
  if len(d[0])==len(d[1])==len(d[2]):
    # d = np.stack( ( fourier(d[0]) , fourier(d[1]) , fourier(d[2]) ) ) 
    x.append(fourier(d[0]))
    y.append(fourier(d[1]))
    z.append(fourier(d[2]))
    labels.append(labelss.iloc[i][1])

In [ ]:
x = np.load('/content/drive/MyDrive/final_test_6000/x_test.npy')
y = np.load('/content/drive/MyDrive/final_test_6000/y_test.npy')
z = np.load('/content/drive/MyDrive/final_test_6000/z_test.npy')
# labels = np.load('/content/drive/MyDrive/plot_signal/labels1_fft2000.pkl.npy')

In [ ]:
x.shape

In [ ]:
xyz = np.transpose(np.stack((x,y,z)) , [1,2,0])
xyz.shape

In [ ]:
xyz=xyz[:,0:200,:]

In [ ]:
xyz.shape

# Load data

In [ ]:
x1 = np.load('/content/drive/MyDrive/data_0-200hz/x600.npy')
y1 = np.load('/content/drive/MyDrive/data_0-200hz/y600.npy')
z1 = np.load('/content/drive/MyDrive/data_0-200hz/z600.npy')
labels1 = np.load('/content/drive/MyDrive/data_0-200hz/labels.npy')

In [ ]:
a = pd.read_csv('/content/sample_submission.csv')
a.iloc[0]['filename']

In [ ]:
x2 = np.load('/content/drive/MyDrive/data_0-200hz/x1200.npy')
y2 = np.load('/content/drive/MyDrive/data_0-200hz/y1200.npy')
z2 = np.load('/content/drive/MyDrive/data_0-200hz/z1200.npy')
labels2 = np.load('/content/drive/MyDrive/data_0-200hz/labels.npy')

In [ ]:
# print(x1.shape,x2.shape)

In [ ]:
# xyz1 = np.transpose(np.stack((x1,y1,z1)) , [1,2,3,0] )
# xyz1.shape

In [ ]:
xyz1 = np.transpose(np.stack((x1,y1,z1)) , [1,2,0])
xyz2 = np.transpose(np.stack((x2,y2,z2)) , [1,2,0])

In [ ]:
xyz = np.concatenate((xyz1, xyz2), axis=0)
xyz.shape

In [ ]:
labels = np.concatenate((labels1, labels2), axis=0)
labels.shape

# Onehot (multitask learning)

In [ ]:
readme = ['Normal', 'UB_1P', 'UB_2P', 'UB_3P', 'MA_P16', 'MA_P20', 'MA_P46', 'MP_N13', 'MP_N16',
          'MP_N18', 'MP_N24', 'MA_P16+UB_1P', 'MA_P16+UB_2P', 'MA_P16+UB_3P', 'MA_P20+UB_1P',
          'MA_P20+UB_2P', 'MA_P20+UB_3P', 'MA_P46+UB_1P', 'MA_P46+UB_2P', 'MA_P46+UB_3P', 'MP_N13+UB_1P', 'MP_N13+UB_2P', 'MP_N13+UB_3P', 
          'MP_N16+UB_1P', 'MP_N16+UB_2P', 'MP_N16+UB_3P', 'MP_N18+UB_1P', 'MP_N18+UB_2P', 'MP_N18+UB_3P', 'Mixcase']
UP = ['no', 'UB_1P', 'UB_2P', 'UB_3P']
MA = ['no', 'MA_P16', 'MA_P20', 'MA_P46']
MP = ['no', 'MP_N13', 'MP_N16','MP_N18', 'MP_N24']
fail2index = { w :i for i, w in enumerate(readme)}
index2fail = { i :w for i, w in enumerate(readme)}

In [ ]:
def onehotUP(label):
  a = np.zeros(4)
  a[label]=1
  return a
def onehotMA(label):
  a = np.zeros(4)
  i = (label%4) +1
  a[i] = 1
  return a
def onehotMP(label):
  a = np.zeros(5)
  i = (label%7) +1
  a[i] = 1
  return a
def makeonehot(fail):
  fail = fail.split('+')
  yup=[]
  yma=[]
  ymp=[]
  for f in fail:
    if f in fail2index:
      if fail2index[f]>=1 and fail2index[f]<=3: yup = onehotUP(fail2index[f])
      if fail2index[f]>=4 and fail2index[f]<=6: yma = onehotMA(fail2index[f])
      if fail2index[f]>=7 and fail2index[f]<=10: ymp = onehotMP(fail2index[f])
  if len(yup)==0: yup = np.array([1,0,0,0], dtype="float32")
  if len(yma)==0: yma = np.array([1,0,0,0], dtype="float32")
  if len(ymp)==0: ymp = np.array([1,0,0,0,0], dtype="float32")
  return yup, yma, ymp
def decode_up(array): return UP[np.argmax(array)]
def decode_ma(array): return MA[np.argmax(array)]
def decode_mp(array): return MP[np.argmax(array)]
fail = 'Normal'
a,b,c = makeonehot(fail)
print( a,b,c)
print(decode_up(a),decode_ma(b),decode_mp(c))

In [ ]:
# Y = []
# for i in y:
#   a=np.zeros(30)
#   a[i] = 1
#   Y.append(a)
# y = np.array(Y)
# # word2index = { w :i + 2 for i, w in enumerate(list(total_word))}

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split( xyz , labels, test_size=0.15)

In [ ]:
yUP_train=[]
yMA_train=[]
yMP_train=[]
for label in y_train:
  fail = index2fail[label]
  yup, yma, ymp = makeonehot(fail)
  yUP_train.append(yup)
  yMA_train.append(yma)
  yMP_train.append(ymp)
yUP_test=[]
yMA_test=[]
yMP_test=[]
for label in y_test:
  fail = index2fail[label]
  yup, yma, ymp = makeonehot(fail)
  yUP_test.append(yup)
  yMA_test.append(yma)
  yMP_test.append(ymp)

# Auto encoder conv1D

In [ ]:
xyz1 = xyz[10:,:,:]

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers , models
from tensorflow.keras.layers import Flatten, Dense, Conv1D, MaxPool1D, Dropout, BatchNormalization, Conv1DTranspose
from tensorflow.keras.constraints import max_norm
batch_size = 150
no_epochs = 50
validation_split = 0.2
verbosity = 1
max_norm_value = 2.0
input_shape = (6299,3)
input_layer = layers.Input(input_shape)
con1d = Conv1D(128, kernel_size=3, kernel_constraint=max_norm(max_norm_value), activation='relu', kernel_initializer='he_uniform')(input_layer)
con1d = Conv1D(64, kernel_size=3, kernel_constraint=max_norm(max_norm_value), activation='relu', kernel_initializer='he_uniform')(con1d)
# con1d3 = Conv1D(64, kernel_size=3, kernel_constraint=max_norm(max_norm_value), activation='relu', kernel_initializer='he_uniform')(con1d2)

# con1d4 = Conv1DTranspose(64, kernel_size=3, kernel_constraint=max_norm(max_norm_value), activation='relu', kernel_initializer='he_uniform')(con1d3)
con1d = Conv1DTranspose(64, kernel_size=3, kernel_constraint=max_norm(max_norm_value), activation='relu', kernel_initializer='he_uniform')(con1d)
con1d = Conv1DTranspose(128, kernel_size=3, kernel_constraint=max_norm(max_norm_value), activation='relu', kernel_initializer='he_uniform')(con1d)

output = Conv1D(3, kernel_size=3, kernel_constraint=max_norm(max_norm_value), activation='sigmoid', padding='same')(con1d)
auto_model = models.Model(inputs=input_layer , outputs= output)
auto_model.summary()

In [ ]:
auto_model.compile(optimizer='adam', loss='mse') #loss='binary_crossentropy'
auto_model.fit( xyz1 , xyz1 ,
                epochs=no_epochs,
                batch_size=batch_size,
                validation_split=validation_split)

In [ ]:
pred = auto_model.predict(xyz[0:5])
pred.shape

In [ ]:
xyz[0][:,0]

In [ ]:
pred[0][:,0]

In [ ]:
X = xyz[0][:,0]
sr = 51200
freq = fftfreq(X.size, 1/sr)
plt.subplot(2,1,1)
plt.plot(freq,X)
# plt.xlim(0,200)
# plt.ylim(0,200)

X = pred[0][:,0]
sr = 51200
freq = fftfreq(X.size, 1/sr)
plt.subplot(2,1,2)
plt.plot(freq,X)
plt.xlim(0,200)
plt.ylim(0,200)

In [ ]:
auto_model.save('/content/drive/MyDrive/model_signal/autoencoder1.h5')

# CNN1D multitask

In [ ]:
import tensorflow as tf
model = tf.keras.models.load_model('/content/drive/MyDrive/model_signal/conv1d_600_1haft.h5')
model.summary()

In [ ]:
#CNN
import tensorflow as tf
from tensorflow.keras import layers , models
from tensorflow.keras.layers import Flatten, Dense, Conv1D, MaxPool1D, Dropout, BatchNormalization

input_shape = (6250,3)
input_layer = layers.Input(input_shape)

conv1D1 = Conv1D(filters=32, kernel_size=(3,), padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.001))(input_layer)
batch1 = BatchNormalization()(conv1D1)

conv1D2 = Conv1D(filters=32, kernel_size=(3,), padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.001))(batch1)
batch2 = BatchNormalization()(conv1D2)

conv1D3 = Conv1D(filters=16, kernel_size=(3,), padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.001))(batch2)
batch3 = BatchNormalization()(conv1D3)

conv1D4 = Conv1D(filters=8, kernel_size=(3,), padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.001))(batch3)
batch4 = BatchNormalization()(conv1D4)

pool = MaxPool1D(pool_size=(3,), strides=2, padding='same')(batch4)
drop = Dropout(0.5)(pool)
flat = Flatten()(pool)

dense1 = Dense(units = 128, activation=tf.keras.layers.LeakyReLU(alpha=0.001))(flat)
dense2 = Dense(units = 64, activation=tf.keras.layers.LeakyReLU(alpha=0.001))(dense1)
dense3 = Dense(units = 32, activation=tf.keras.layers.LeakyReLU(alpha=0.001))(dense2)

outputUB = Dense(units = len(UP), activation='softmax')(dense3)
outputMA = Dense(units = len(MA), activation='softmax')(dense3)
outputMP = Dense(units = len(MP), activation='softmax')(dense3)
model = models.Model(inputs=input_layer, outputs=[outputUB , outputMA , outputMP])

In [ ]:
loss = tf.keras.losses.categorical_crossentropy
optimizer = tf.keras.optimizers.Adam(lr=1e-5)
model.compile(optimizer=optimizer, loss = loss, metrics=['accuracy'])
model.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import os

modelPath = '/content/model{epoch:02d}.hdf5'
# checkpoint = ModelCheckpoint( # set model saving checkpoints
#     modelPath, # set path to save model weights
#     monitor='val_loss', # set monitor metrics
#     verbose=1, # set training verbosity
#     save_best_only=True, # set if want to save only best weights
#     save_weights_only=False, # set if you want to save only model weights
#     mode='auto', # set if save min or max in metrics
#     period=1 # interval between checkpoints
#     )
earlystopping = EarlyStopping(
    monitor='val_loss', # set monitor metrics
    min_delta=0.001, # set minimum metrics delta
    patience=8, # number of epochs to stop training
    restore_best_weights=True, # set if use best weights or last weights
    )
callbacksList = [earlystopping] # build callbacks list

model_history = model.fit( x_train , [np.array(yUP_train), np.array(yMA_train), np.array(yMP_train)], epochs=100, 
                  batch_size = 64, 
                validation_data = ( x_test, [np.array(yUP_test), np.array(yMA_test), np.array(yMP_test)] ), 
                callbacks=callbacksList)

In [ ]:
model.save('/content/drive/MyDrive/model_signal/conv1d_all_shape49.h5')

# Submit

In [ ]:
sample_submission = pd.read_csv('/content/drive/MyDrive/sample_submission.csv')
sample_submission

In [ ]:
# from tqdm import tqdm
# pathtest = '/content/drive/MyDrive/maintain/test_data/'
# x_test = []
# y_test = []
# z_test = []
# labels = []
# for i in tqdm(range(len(sample_submission.iloc[:]))):
#   d = pd.read_csv(pathtest + sample_submission.iloc[i]['filename'] , header = None)
#   if len(d[0])==len(d[1])==len(d[2]):
#     x.append(fourier(d[0]))
#     y.append(fourier(d[1]))
#     z.append(fourier(d[2]))

In [ ]:
# np.save('/content/drive/MyDrive/data_signal/x1_fft2000.pkl', x)
# np.save('/content/drive/MyDrive/data_signal/y1_fft2000.pkl', y)
# np.save('/content/drive/MyDrive/data_signal/z1_fft2000.pkl', z)

In [ ]:
import tensorflow as tf
model = tf.keras.models.load_model('/content/drive/MyDrive/model_signal/conv1d_600_1haft.h5')
model.summary()

In [ ]:
x_test = np.load('/content/drive/MyDrive/x_test.npy')
y_test = np.load('/content/drive/MyDrive/y_test.npy')
z_test = np.load('/content/drive/MyDrive/z_test.npy')

In [ ]:
xyz_test = np.transpose(np.stack((x_test,y_test,z_test)) , [1,2,0])
xyz_test.shape

In [ ]:
pred = model.predict(xyz_test)
pred[0].shape

In [ ]:
from tqdm import tqdm
predict = []
for i in tqdm(range(1800)):
  a,b,c = pred[0][i], pred[1][i], pred[2][i] 
  # print(a,b,c)
  pr = []
  p = [decode_up(a),decode_ma(b),decode_mp(c)]
  # print(p)
  for j in p:
    if j!='no': pr.append(j)
  if len(pr)>0 and len(pr)<3:
    pr = '+'.join(sorted(pr))
  elif len(pr)==3:
    pr = 'Mixcase'
  else:
    pr = 'Normal'
  if pr in fail2index: predict.append(fail2index[pr])
  else: predict.append(fail2index['Normal'])
print(predict)

In [ ]:
df = pd.DataFrame({ 'filename':sample_submission['filename'], 'code':predict })
df.to_csv('/content/submit.csv')